In [5]:
import holdem_hand_evaluator
import random as rand
import time
import copy as cp

holdem_hand_evaluator.evaluate_hand("7h7c7d7sAc9c8c")
#bigger is better

28743

In [6]:
class Player():
    def __init__(self):
        self.cards = None
        self.stack = 10**3
    def bet(self, amount):
        if amount > self.stack:
            amount = self.stack
        self.stack -= amount
        return amount, self.stack

In [15]:
class dealer():
    def __init__(self, Players):
        vals = [str(i) for i in range(2,10)] + ['T', 'J', 'Q', 'K', 'A']
        suits = ['h', 'c', 'd', 's']
        self.deck = [j + i for i in suits for j in vals]
        self.Players = Players
        self.played = []
        self.pot = 0
    
    def take_card(self):
        card = rand.choice(self.deck)
        self.deck.remove(card)
        return card
    
    def preflop(self):
        for player in self.Players:
            player.cards = [self.take_card(), self.take_card()]
        
    def flop(self):
        cs = [self.take_card(), self.take_card(), self.take_card()]
        self.played = cs
        return cs

    def turn(self):
        cs = [self.take_card()]
        self.played += cs
        return cs

    def river(self):
        cs = [self.take_card()]
        self.played += cs
        return cs
    
    def betting(self):
        playrs = [[p, 0] for p in self.Players]
        #just changed so error
        previous = playrs[0][1]
        i=0
        runitback = False
        while i < len(playrs):
            p = playrs[i][0]
            commit=playrs[i][1]
            if commit < previous or previous == 0:
                bet = int(input(f'what would you like to bet with {p.cards} and {p.stack}, having commited to this round {commit}. To call or check, please bet {previous-commit} ? '))
                while bet+commit < previous or previous<bet+commit<2*previous:
                    if bet<previous:
                        bet = int(input(f'to call you must bet at least {previous-commit}'))
                    else:
                        bet = int(input(f'To min raise, you must bet at least {2*previous-commit}'))
                playrs[i][1] += bet
                p.bet(bet)
                self.pot += bet
            
            if playrs[i][1] > previous:
                #if raise --> we do another round of betting
                previous = playrs[i][1]
                runitback = True
            
            i+=1
            
            if i == len(playrs) and runitback:
                runitback = False
                i = 0  
    
    def award_winner(self):
        scores = []
        for player in self.Players:
            scores.append(holdem_hand_evaluator.evaluate_hand(''.join(player.cards + self.played)))
        winner_index = scores.index(max(scores))
        print(f'winning hand: {self.Players[winner_index].cards}')
        self.Players[winner_index].stack += self.pot
        return f'new stack {self.Players[winner_index].stack}' 

In [16]:
def run_game(x):
    ps = [Player() for i in range(x)]       
    d = dealer(ps)
    d.preflop()
    for p in ps:
        print(p.cards)
    time.sleep(1)
    d.betting()
    print(d.pot)
    time.sleep(1)
    print(d.flop())
    d.betting()
    print(d.pot)
    time.sleep(1)
    print(d.turn())
    d.betting()
    print(d.pot)
    time.sleep(1)
    print(d.river())
    d.betting()
    print(d.pot)
    print('-------------------------------------')
    d.award_winner()
        


In [17]:
run_game(2)

['Ts', 'Ks']
['Ac', 'Qh']
40
['7s', '6h', 'As']
160
['4h']
200
['8h']
320
-------------------------------------
winning hand: ['Ac', 'Qh']
